In [ ]:
%matplotlib inline
import tables
import numpy
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib

import glob
import sys
import matplotlib.gridspec as gridspec
import CalculateMeanWinds


# Helper functions

In [ ]:
def readafile(fname):
    '''
    Function written by nicolls to read
    '''
    h5file=tables.open_file(fname)
    output={}
    for group in h5file.walk_groups("/"):
        output[group._v_pathname]={}
        for array in h5file.list_nodes(group, classname = 'Array'):
            output[group._v_pathname][array.name]=array.read()
    h5file.close()

    return output

def WeightedMeanNaN(x, dx):
    xdx = x/dx
    # bevington 4.17
    weightedMean = numpy.nansum(x/(dx**2), axis=0)/numpy.nansum(1./dx**2, axis=0)
    # bevington 4.19
    weightedVar = 1./numpy.nansum((1./dx**2), axis=0)
    weightedStd = numpy.sqrt(weightedVar)
    n = [numpy.where(numpy.isfinite(xdx[:,i]))[0].shape[0] for i in range(xdx.shape[1])]
    # print n
    return weightedMean, weightedStd, n

def MakeQuiverFigures(DecimalHours, ZonalWinds,MeridWinds,radius=0.5):
    # can now validate using testmltplots.ipynb
    timeInDegrees = 180-360.*(DecimalHours/24.)
#     print 'timeInDegrees', DecimalHours,timeInDegrees
    theta = numpy.deg2rad(timeInDegrees)
    radii = numpy.ones(theta.shape[0])*radius
    Uin = ZonalWinds
    Vin = MeridWinds
    #test
    # Vin = 0.0
    # Uin = 1.0
    Vprime = -numpy.cos(theta)*Vin + numpy.sin(theta)*Uin
    Uprime = -numpy.sin(theta)*Vin + -numpy.cos(theta)*Uin




    # plt.show()
    return theta,radii, Vprime, Uprime

In [ ]:
def PlotWindsVGeophysicalParams(inDict,KeySelect, outfile):
    nrow = 4
    ncol = 4
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(12,12))
    gs = gridspec.GridSpec(nrow, ncol)
    iplot = 7 # sets the altitude we are interested in
    iloc = 1
    
    keyList = sorted(inDict.keys())
    print keyList
    
    ikey = 0
    for irow in range(0,nrow,1):
        print ikey, keyList[ikey]
        mltTimeDict = inDict[keyList[ikey]][KeySelect]
        for icol in range(0,ncol,1):
            print 'irow,icol',totalNumPlots,irow,icol
            if icol==0:
                ax = plt.subplot(gs[irow,icol],projection='polar')


                # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
                # for irow in range(nrow):
                #     for icol in range(ncol):
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                              mltTimeDict['MeanZonalWinds'][0,:,iplot],\
                                                              mltTimeDict['MeanMeridWinds'][0,:,iplot], \
                                                              radius=0.5)

                ax.quiver(theta, radii, Uprime, Vprime,color='red')
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                              mltTimeDict['MeanZonalFlow'][1,:],\
                                                              mltTimeDict['MeanMeridFlow'][1,:], \
                                                              radius=1.0)

                ax.quiver(theta, radii, Uprime, Vprime,color='blue')
                ax.set_theta_direction(-1)
                ax.set_theta_zero_location('N')
                ax.set_rlim([0,1.5])
                ax.set_rticks([])
                ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
                ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
                ax.set_xticklabels(['00', '18', '12', '06'])
                ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][iplot])


            # do the electron density vs time
            if icol==1:
                ax = plt.subplot(gs[irow,icol])

                # make a little helper loop
                Xmean = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                Xstd = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                for itime in range(mltTimeDict['DecimalHoursTimeGrid'].shape[0]):
                    X = mltTimeDict['NeMeanRaw'][itime,iplot]
                    X = numpy.log10(X[~numpy.isnan(X)])
                    Xmean[itime] = numpy.mean(X)
                    Xstd[itime] = numpy.std(X)
                Xmin = 11.0
                Xmax = 12.0
                ax.errorbar(mltTimeDict['DecimalHoursTimeGrid'],Xmean,yerr=Xstd, color='red', ecolor='blue')
                ax.set_ylim([Xmin,Xmax])
                ax.set_xlim([0,24])
                ax.set_xticks([0,6,12,18,24])
                ax.set_title('Ne, %s'%keyList[ikey])
                ax.grid()

            # AE
            if icol == 2:
                ax = plt.subplot(gs[irow,icol])

                # make a little helper loop
                Xmean = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                Xstd = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                for itime in range(mltTimeDict['DecimalHoursTimeGrid'].shape[0]):
                    X = mltTimeDict['AERaw'][itime,:]
                    X = X[~numpy.isnan(X)]
                    Xmean[itime] = numpy.mean(X)
                    Xstd[itime] = numpy.std(X)
                Xmin = 0.
                Xmax = 1000.0
                ax.errorbar(mltTimeDict['DecimalHoursTimeGrid'],Xmean,yerr=Xstd, color='red', ecolor='blue')
                ax.set_ylim([Xmin,Xmax])
                ax.set_xlim([0,24])
                ax.set_xticks([0,6,12,18,24])
                ax.grid()
                ax.set_title('AE')

            # KP
            if icol == 3:
                ax = plt.subplot(gs[irow,icol])

                # make a little helper loop
                Xmean = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                Xstd = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                for itime in range(mltTimeDict['DecimalHoursTimeGrid'].shape[0]):
                    X = mltTimeDict['KPRaw'][itime,:]
                    X = X[~numpy.isnan(X)]
                    X = (X-3.)/9.0
                    X[X<0] = 0.
                    Xmean[itime] = numpy.mean(X)
                    Xstd[itime] = numpy.std(X)
                Xmin = 0.
                Xmax = 9.0
                ax.errorbar(mltTimeDict['DecimalHoursTimeGrid'],Xmean,yerr=Xstd, color='red', ecolor='blue')
                ax.set_ylim([Xmin,Xmax])
                ax.set_xlim([0,24])
                ax.set_xticks([0,6,12,18,24])
                ax.grid()
                ax.set_title('KP')
        ikey=ikey+1
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    plt.tight_layout()
    f.savefig(outfile)

In [ ]:
def PlotWindsVNe(inDict,KeySelect,ialt1,ialt2, outfile):
    nrow = 4
    ncol = 4
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(12,12))
    gs = gridspec.GridSpec(nrow, ncol)
    iplot1 = ialt # sets the altitude we are interested in
    iplot2 = 7
    iloc = 1
    
    keyList = sorted(inDict.keys())
    
    
    ikey = 0
    for irow in range(0,nrow,1):
        print ikey, keyList[ikey]
        mltTimeDict = inDict[keyList[ikey]][KeySelect]
        for icol in range(0,ncol,1):
            print 'irow,icol',totalNumPlots,irow,icol
            if icol==0:
                ax = plt.subplot(gs[irow,icol],projection='polar')


                # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
                # for irow in range(nrow):
                #     for icol in range(ncol):
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                              mltTimeDict['MeanZonalWinds'][0,:,iplot1],\
                                                              mltTimeDict['MeanMeridWinds'][0,:,iplot1], \
                                                              radius=0.5)

                ax.quiver(theta, radii, Uprime, Vprime,color='red')
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                              mltTimeDict['MeanZonalFlow'][1,:],\
                                                              mltTimeDict['MeanMeridFlow'][1,:], \
                                                              radius=1.0)

                ax.quiver(theta, radii, Uprime, Vprime,color='blue')
                ax.set_theta_direction(-1)
                ax.set_theta_zero_location('N')
                ax.set_rlim([0,1.5])
                ax.set_rticks([])
                ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
                ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
                ax.set_xticklabels(['00', '18', '12', '06'])
                ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][iplot1])


            # do the electron density vs time
            if icol==1:
                ax = plt.subplot(gs[irow,icol])

                # make a little helper loop
                Xmean = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                Xstd = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                for itime in range(mltTimeDict['DecimalHoursTimeGrid'].shape[0]):
                    X = mltTimeDict['NeMeanRaw'][itime,iplot1]
                    X = numpy.log10(X[~numpy.isnan(X)])
                    Xmean[itime] = numpy.mean(X)
                    Xstd[itime] = numpy.std(X)
                Xmin = 11.0
                Xmax = 12.0
                ax.errorbar(mltTimeDict['DecimalHoursTimeGrid'],Xmean,yerr=Xstd, color='red', ecolor='blue')
                ax.set_ylim([Xmin,Xmax])
                ax.set_xlim([0,24])
                ax.set_xticks([0,6,12,18,24])
                ax.set_title('Ne, %s'%keyList[ikey])
                ax.grid()

            if icol==2:
                ax = plt.subplot(gs[irow,icol],projection='polar')


                # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
                # for irow in range(nrow):
                #     for icol in range(ncol):
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                              mltTimeDict['MeanZonalWinds'][0,:,iplot2],\
                                                              mltTimeDict['MeanMeridWinds'][0,:,iplot2], \
                                                              radius=0.5)

                ax.quiver(theta, radii, Uprime, Vprime,color='red')
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                              mltTimeDict['MeanZonalFlow'][1,:],\
                                                              mltTimeDict['MeanMeridFlow'][1,:], \
                                                              radius=1.0)

                ax.quiver(theta, radii, Uprime, Vprime,color='blue')
                ax.set_theta_direction(-1)
                ax.set_theta_zero_location('N')
                ax.set_rlim([0,1.5])
                ax.set_rticks([])
                ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
                ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
                ax.set_xticklabels(['00', '18', '12', '06'])
                ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][iplot2])


            # do the electron density vs time
            if icol==3:
                ax = plt.subplot(gs[irow,icol])

                # make a little helper loop
                Xmean = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                Xstd = numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0])
                for itime in range(mltTimeDict['DecimalHoursTimeGrid'].shape[0]):
                    X = mltTimeDict['NeMeanRaw'][itime,iplot2]
                    X = numpy.log10(X[~numpy.isnan(X)])
                    Xmean[itime] = numpy.mean(X)
                    Xstd[itime] = numpy.std(X)
                Xmin = 11.0
                Xmax = 12.0
                ax.errorbar(mltTimeDict['DecimalHoursTimeGrid'],Xmean,yerr=Xstd, color='red', ecolor='blue')
                ax.set_ylim([Xmin,Xmax])
                ax.set_xlim([0,24])
                ax.set_xticks([0,6,12,18,24])
                ax.set_title('Ne, %s'%keyList[ikey])
                ax.grid()
        ikey=ikey+1
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    plt.tight_layout()
    f.savefig(outfile)

In [ ]:
def PlotWindsVYear(inDict,KeySelect,inalt, outfile):
    nrow = len(inDict)
    ncol = len(inalt)+1
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(8.5,11))
    gs = gridspec.GridSpec(nrow, ncol)
#     iplot1 = ialt # sets the altitude we are interested in
#     iplot2 = 7
    iloc = 1
    
    keyList = sorted(inDict.keys())
    
    
    ikey = 0
    for irow in range(0,nrow,1):
        print ikey, keyList[ikey]
        mltTimeDict = inDict[keyList[ikey]][KeySelect]
        for icol in range(len(inalt)):
            print 'irow,icol',totalNumPlots,irow,icol
            
            ax = plt.subplot(gs[irow,icol],projection='polar')


            # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
            # for irow in range(nrow):
            #     for icol in range(ncol):
            ialt = inalt[icol]
            theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalWinds'][0,:,ialt],\
                                                          mltTimeDict['MeanMeridWinds'][0,:,ialt], \
                                                          radius=0.5)

            ax.quiver(theta, radii, Uprime, Vprime,color='red')
            theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalFlow'][1,:],\
                                                          mltTimeDict['MeanMeridFlow'][1,:], \
                                                          radius=1.0)

            ax.quiver(theta, radii, Uprime, Vprime,color='blue')
            ax.set_theta_direction(-1)
            ax.set_theta_zero_location('N')
            ax.set_rlim([0,1.5])
            ax.set_rticks([])
            ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
            ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
            ax.set_xticklabels(['00', '18', '12', '06'])
            ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][ialt])


            
        ikey=ikey+1
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    gs.tight_layout(f)
    f.savefig(outfile)

In [ ]:
def PlotFlowsAndWindsVYear(inDict,KeySelect,inalt, outfile):
    nrow = len(inDict)
    ncol = len(inalt)+1
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(7,9))
    gs = gridspec.GridSpec(nrow, ncol)
#     iplot1 = ialt # sets the altitude we are interested in
#     iplot2 = 7
    iloc = 1
    
    keyList = sorted(inDict.keys())
    
    # scaling of the vectors
    scale = 350.
    Wscale = 250.
    
    ikey = 0
    for irow in range(0,nrow,1):
        print ikey, keyList[ikey]
        mltTimeDict = inDict[keyList[ikey]][KeySelect]
        for icol in range(ncol):
            print 'irow,icol',totalNumPlots,irow,icol
            rticks = numpy.cos(numpy.deg2rad(numpy.arange(90.,40.,-10)))
            rPFISR = numpy.cos(numpy.deg2rad(68.))
            # plot the flows
            if icol == 0:
                ax = plt.subplot(gs[irow,icol],projection='polar')
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalFlow'][1,:],\
                                                          mltTimeDict['MeanMeridFlow'][1,:], \
                                                          radius=rPFISR)

                Q = ax.quiver(theta, radii, Uprime, Vprime,color='blue', units='xy', scale=scale, pivot='tail')
#                 ax.quiver(0,0.2,100.,100., color='cyan', units='xy', scale=10.)
                
#                 theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
#                                                           mltTimeDict['MeanZonalFlowFregion'][1,:],\
#                                                           mltTimeDict['MeanMeridFlowFregion'][1,:], \
#                                                           radius=rPFISR+0.1)
#                 ax.quiver(theta, radii, Uprime, Vprime,color='red')
                ax.set_theta_direction(-1)
                ax.set_theta_zero_location('N')
                ax.set_rlim([0,0.5])
                ax.set_rticks(rticks)
                ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
                ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
                ax.set_xticklabels(['00', '18', '12', '06'])
                ax.set_ylabel('%s \n \n'%keyList[ikey][0:4], fontsize=14)
                ax.quiverkey(Q, 0.95, 0.95, 200, r'$200 \frac{m}{s}$', labelpos='E')
                ax.set_xlabel('300 km', fontsize=16)
    
            
            if icol != 0:
                ax = plt.subplot(gs[irow,icol],projection='polar')

                # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
                # for irow in range(nrow):
                #     for icol in range(ncol):
                ialt = inalt[icol-1]
                print mltTimeDict['MeanAltitude'][ialt]
                
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalWinds'][0,:,ialt],\
                                                          mltTimeDict['MeanMeridWinds'][0,:,ialt], \
                                                          radius=rPFISR)

                Q = ax.quiver(theta, radii, Uprime, Vprime,color='red', units='xy', scale=Wscale, pivot='tail')
                ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][ialt], fontsize=14)
                print 'MaxMeridonalWind', numpy.nanmax(mltTimeDict['MeanMeridWinds'][0,:,ialt])
                ax.quiverkey(Q, 0.95, 0.95, 100, r'$100 \frac{m}{s}$', labelpos='E')
            
#             ax.quiver(theta, radii, Uprime, Vprime,color='blue')
            ax.set_theta_direction(-1)
            ax.set_theta_zero_location('N')
            ax.set_rlim([0,0.5])
            ax.set_rticks(rticks)
            ax.set_yticklabels([])
#             ax.set_rlabel_position([)
            ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
#             ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
#             ax.set_xticklabels(['00', '18', '12', '06'])
            ax.set_xticks([-numpy.pi,3.*numpy.pi/4., numpy.pi/2., numpy.pi/4, 0.,\
                           -numpy.pi/4., -numpy.pi/2., -3.*numpy.pi/4. ]) # 00, 03, 06, 09, 12, 15, 18 ,21
            ax.set_xticklabels(['00','','06','','12','','18', ''])
            

                


            
        ikey=ikey+1
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    gs.tight_layout(f)
    f.savefig(outfile)

In [ ]:
def PlotFlowsAndWindsVYearMovie(inDict,KeySelect,inalt, outfile):
    nrow = len(inDict)
    ncol = len(inalt)+1
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(7,4))
    gs = gridspec.GridSpec(nrow, ncol)
#     iplot1 = ialt # sets the altitude we are interested in
#     iplot2 = 7
    iloc = 1
    
    keyList = sorted(inDict.keys())
    
    # scaling of the vectors
    scale = 350.
    Wscale = 250.
    
    
    ikey = 0
    for irow in range(0,nrow,1):
        print ikey, keyList[ikey]
        mltTimeDict = inDict[keyList[ikey]][KeySelect]
        for icol in range(ncol):
            print 'irow,icol',totalNumPlots,irow,icol
            rticks = numpy.cos(numpy.deg2rad(numpy.arange(90.,40.,-10)))
            rPFISR = numpy.cos(numpy.deg2rad(68.))
            # plot the flows
            if icol == 0:
                ax = plt.subplot(gs[irow,icol],projection='polar')
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalFlow'][1,:],\
                                                          mltTimeDict['MeanMeridFlow'][1,:], \
                                                          radius=rPFISR)

#                 ax.quiver(theta, radii, Uprime, Vprime,color='blue')
                Q = ax.quiver(theta, radii, Uprime, Vprime,color='blue', units='xy', scale=scale, pivot='tail')
                ax.quiverkey(Q, 0.95, 0.95, 200, r'$200 \frac{m}{s}$', labelpos='E')
                
#                 theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
#                                                           mltTimeDict['MeanZonalFlowFregion'][1,:],\
#                                                           mltTimeDict['MeanMeridFlowFregion'][1,:], \
#                                                           radius=rPFISR+0.1)
#                 ax.quiver(theta, radii, Uprime, Vprime,color='red')
                ax.set_theta_direction(-1)
                ax.set_theta_zero_location('N')
                ax.set_rlim([0,0.45])
                ax.set_rticks(rticks)
                ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
                ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
                ax.set_xticklabels(['00', '18', '12', '06'])
                ax.set_xlabel('300 km', fontsize=16)
                ax.set_ylabel('%s \n \n'%keyList[ikey][0:6], fontsize=14)
                
    
            
            if icol != 0:
                ax = plt.subplot(gs[irow,icol],projection='polar')

                # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
                # for irow in range(nrow):
                #     for icol in range(ncol):
                ialt = inalt[icol-1]
                print mltTimeDict['MeanAltitude'][ialt]
                
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalWinds'][0,:,ialt],\
                                                          mltTimeDict['MeanMeridWinds'][0,:,ialt], \
                                                          radius=rPFISR)

#                 ax.quiver(theta, radii, Uprime, Vprime,color='red')
                Q = ax.quiver(theta, radii, Uprime, Vprime,color='red', units='xy', scale=Wscale, pivot='tail')
                ax.quiverkey(Q, 0.95, 0.95, 100, r'$100 \frac{m}{s}$', labelpos='E')
#                 ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][ialt])
                ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][ialt], fontsize=14)
                
            
#             ax.quiver(theta, radii, Uprime, Vprime,color='blue')
            ax.set_theta_direction(-1)
            ax.set_theta_zero_location('N')
            ax.set_rlim([0,0.45])
            ax.set_rticks(rticks)
            ax.set_yticklabels([])
#             ax.set_rlabel_position([)
            ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
#             ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
#             ax.set_xticklabels(['00', '18', '12', '06'])
            ax.set_xticks([-numpy.pi,3.*numpy.pi/4., numpy.pi/2., numpy.pi/4, 0.,\
                           -numpy.pi/4., -numpy.pi/2., -3.*numpy.pi/4. ]) # 00, 03, 06, 09, 12, 15, 18 ,21
            ax.set_xticklabels(['00','','06','','12','','18', ''])
            

                


            
        ikey=ikey+1
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    gs.tight_layout(f)
    f.savefig(outfile)

# Testing main Figure

In [ ]:
# fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/200806_Winds.v0.4.2018.03.15.test.h5',
#           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/200906_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201006_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201106_Winds.v0.4.2018.03.15.test.h5']
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofile.ini'
# CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
# TmpDict = CalcMeanWinds.main(fnames)
# KeySelect = 'mlt'
# PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2008_2011_WindsV3Altitudes.10092018.png')

# fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201206_Winds.v0.4.2018.03.15.test.h5',
#           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201306_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201406_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201506_Winds.v0.4.2018.03.15.test.h5']

fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.2.2018.10.15/MonthlyWinds/201106_Winds.v0.4.2.2018.10.15.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.2.2018.10.15/MonthlyWinds/201206_Winds.v0.4.2.2018.10.15.h5']
#           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201306_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201406_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201506_Winds.v0.4.2018.03.15.test.h5']
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
TmpDict = CalcMeanWinds.main(fnames)
KeySelect = 'mlt'
# PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
PlotFlowsAndWindsVYear(TmpDict,'mlt',[7,5], 'foo.png')

In [ ]:
# print TmpDict['201106']['mlt']['MeanMeridFlowFregion']
# print TmpDict['201106']['mlt']['MeanMeridFlow']

plt.figure()
plt.plot(TmpDict['201106']['mlt']['MeanMeridFlowFregion'][2,:], 'r-')
plt.plot(TmpDict['201106']['mlt']['MeanMeridFlow'][2,:], 'b-')

plt.figure()
plt.plot(TmpDict['201106']['mlt']['MeanZonalFlowFregion'][2,:], 'r-')
plt.plot(TmpDict['201106']['mlt']['MeanZonalFlow'][2,:], 'b-')

plt.figure()


In [ ]:
plt.figure()
# zonal E field
plt.plot(TmpDict['201106']['mlt']['MeanMeridFlowFregion'][2,:]*0.5/1e4, 'r-')
plt.plot(TmpDict['201106']['mlt']['MeanZonalE'][2,:], 'b-')

plt.figure()
plt.plot(-TmpDict['201106']['mlt']['MeanZonalFlowFregion'][2,:]*0.5/1e4, 'r-')
plt.plot(TmpDict['201106']['mlt']['MeanMeridE'][2,:], 'b-')

# Main Figure Generation

In [ ]:

# this config file is the test config file
#configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofileTest.ini'

# this is the config file for the figures in the paper:
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/ConfigFileForPaper_02212019.ini'


fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201006_Winds.v0.4.5.2018.11.20.h5',
            '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201106_Winds.v0.4.5.2018.11.20.h5',
           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201206_Winds.v0.4.5.2018.11.20.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201306_Winds.v0.4.5.2018.11.20.h5'
        ]
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
TmpDict = CalcMeanWinds.main(fnames)
KeySelect = 'mlt'
# PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
PlotFlowsAndWindsVYear(TmpDict,'mlt',[7,5], './FiguresForPaper/2010_2013_PlasmaFlowVWinds3Altitude_MLTvMLat_02212019.pdf')

fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201406_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201506_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201606_Winds.v0.4.5.2018.11.20.h5']

CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
TmpDict = CalcMeanWinds.main(fnames)
KeySelect = 'mlt'
# PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
PlotFlowsAndWindsVYear(TmpDict,'mlt',[7,5], './FiguresForPaper/2014_2016_PlasmaFlowVWinds3Altitude_MLTvMLat_02212019.pdf')

# Movie Generation


In [ ]:
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofile.ini'
fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201406_Winds.v0.4.2.2018.10.15.h5',
         ]

fnames = sorted(glob.glob('/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/*.h5'))
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)

# print fnames

for i in range(len(fnames)):
    print fnames[i]
    TmpDict = CalcMeanWinds.main([fnames[i]])
#     KeySelect = 'mlt'
# # PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
    PlotFlowsAndWindsVYearMovie(TmpDict,'mlt',[7,5], '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/%03d.png'%i)

In [ ]:
fnames = sorted(glob.glob('/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/*.h5'))
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)

# print fnames

for i in range(len(fnames)):
    print fnames[i]
    TmpDict = CalcMeanWinds.main([fnames[i]])
#     KeySelect = 'mlt'
# # PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
    PlotFlowsAndWindsVYearMovie(TmpDict,'mlt',[7,5], '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/%03d.png'%i)

#/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds